# CRIM Intervals:  Cadences

### Reminders:

#### Import Music Files

* If you are exploring pieces from CRIM, importing simply involves providing the CRIM URL of the MEI file:  
    * **`piece = importScore('https://crimproject.org/mei/CRIM_Model_0008.mei')`**

* But you can also use the Notebook with any MEI, MusicXML, or MIDI file of your own. You can easily do this when you run the Notebooks on Jupyter Hub, you will also find a folder called **`Music_Files`**.  Upload the file here, then provide the path to that file: 
    * **`piece = importScore('Music_Files/My_File_Name.mei')`**.  

#### Save outputs as CSV or Excel

* The Jupyter Hub version of these Notebooks also provides a folder called **`saved_csv`**.  You can save **csv** files of any data frame there with this command: 
    * **`notebook_data_frame_name.to_csv('saved_csv/your_file_title.csv')`**.
* If you prefer **Excel** documents (which are better for anything with a complex set of columns or hierarhical index), use **ExcelWriter**.  In the following code, you will need to provide these commands:
    * **`writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')`**
* Now convert your dataframe to Excel
    * **`frame_name.to_excel(writer, sheet_name='Sheet1')`**
* And finally save the new file to the folder here in the Notebook:
    * **`writer.save()`**

Put the following code to a new cell and update the frame_name and file_name:

`writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')` <br>
`frame_name.to_excel(writer, sheet_name='Sheet1')` <br>
`writer.save()` <br>


## A. Import Intervals and Other Code

* The first step is to import all the code required for the Notebook
* **`arrow/run`** or **`Shift + Enter`** in the following cell:

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os

MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


## B. Importing a Piece

### B.1 Import a Piece and Check Title

In [ ]:

piece = importScore('https://crimproject.org/mei/CRIM_Model_0008.mei')
piece.score.metadata.title

## C. Find Cadences with Modular Analysis

Alex Morgan has built a powerful tool that identifies cadences according to the combinations of two-voice **modules** that describe the typical contrapuntal motion between the various **cadential voice functions** (**CVF**) heard in Renaissance polyphony:  **cantizans and tenorizans**, **cantizans and bassiszans**, etc.

The tool uses modular analysis to identify **conjunctions** of these pairs in order to predict cadences of various kinds. But there are many combinations, especially once we consider that voices functions (or roles) can be **displaced** (as when the tenorizans role appears in the Superius part and the cantizans appears in the Tenor part), or through **irregular** motion, and even **interrupted**, as when a voice is suddenly silent. 

You can in fact check all of the cadential voice functions (CVFs) for a given piece below.  But Alex's system also conveniently **labels** the cadences according to **type**, **tone**, **evaded** and also provides information about the relative place within the piece, the adjacent cadences, and many other features, too.

Note:  **Measure** and **Beat** columns are in the body of the table, not at the Index.

**Column Headings Explained**:

* The **Low** and **Tone** columns give the pitches of the **lowest sounding pitch (in any voice) at the perfection**, and the **goal tone of the cantizans** (or altizans if there is no cantizans) respectively.

* **RelLow** is the lowest pitch of each cadence shown as an interval measured against the last pitch in the **Low** column. Likewise, **RelTone** is the cadential tone shown as an interval measured against the last pitch in the **Tone** column.

* The **SinceLast** and **ToNext** columns are the time in quarter notes since the last or to the next cadence.

* The **Progress** column is a relative indication of position in the piece.  **0** is the beginning of the piece; **1.0** is the end of the piece.



Read more via the documentation: **`print(piece.classifyCadences.__doc__)`**

View the **Cadential Voice Function** and **Cadence Label** tables here:  https://github.com/HCDigitalScholarship/intervals/tree/main/intervals/data/cadences/.  These can easily be updated with revised or new cadence types.

### C.1 Classify the Cadences

In [ ]:
cadences = piece.classifyCadences()
col_list = ['Measure', 'Beat', 'CadType', 'Tone','Evaded','LeadingTones', 'Low','RelLow','RelTone','Progress','SinceLast','ToNext']
cadences = cadences[col_list]
cadences

### C.2  Check Voice Functions for all Cadences of a Given Piece

When **return_type** is set to **'functions'** (or just 'f' for short), a table
of the **cadential voice functions** (CVF) is returned. Each CVF is
represented with a single-character label with the meanings as follows:

* `C`: **cantizans motion up a step** (can also be ornamented e.g. Landini)
* `T`: **tenorizans motion down a step** (can be ornamented with anticipations)
* `B`: **bassizans motion up a fourth or down a fifth**
* `A`: **altizans motion, similar to cantizans**, but cadences to a fifth
above a tenorizans instead of an octave  <br><br>
* `L`: **leaping contratenor** motion up an octave at the perfection
* `P`: **plagal bassizans** motion up a fifth or down a fourth <br><br>
* `c`: **evaded cantizans** when it moves to an unexpected note at the perfection
* `t`: **evaded tenorizans** when it goes up by step at the perfection
* `b`: **evaded bassizans** when it goes up by step at the perfection
* `u`: **evaded bassizans** when it goes down by third at the perfection
(there are no evaded labels for the altizans, plagal bassizans leaping
contratenor CVFs) <br><br>
* `x`: evaded **bassizans motion where the voice drops out** at the perfection
* `y`: evaded **cantizans motion where the voice drops out** at the perfection
* `z`: evaded **tenorizans motion where the voice drops out** at the perfection


View the **Cadential Voice Function** and **Cadence Label** tables here: https://github.com/HCDigitalScholarship/intervals/tree/main/intervals/data/cadences/. These can easily be updated with revised or new cadence types.

In [ ]:
cvf = piece.classifyCadences(return_type='f')
piece.detailIndex(cvf, offset=True).fillna('-')


### C.3.  Check for Missed Cadences (One or More Pieces)


* The resulting table lists the **Cadential Voice Functions** (CVF's) for each of the probable cadences that the Cadence Classifier was **unable to categorize** according to the **Cadential Voice Function** and **Cadence Label** tables here: https://github.com/HCDigitalScholarship/intervals/tree/main/intervals/data/cadences/. These can easily be updated with revised or new cadence types.

* See below for additional instructions.

In [ ]:

corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0008.mei']) # add as many pieces as you want here, each in single quotations, separated by commas
cad_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=False)
cvf_dfs = corpus.batch(ImportedPiece.classifyCadences, kwargs={'return_type': 'f'})
missed = []
for i, cad in enumerate(cad_dfs):
    df = cvf_dfs[i].loc[cad.CadType.isnull(), :]
    df = df[df.columns[range(-2, len(df.columns) -2)]]
    df.columns = range(len(df.columns))
    missed.append(df)
result = pd.concat(missed)
rm = piece.detailIndex(result, offset=True)
result_offset_list = rm.index.get_level_values('Offset')
rm

### C.4.  Check the Harmonic Modules of Missing Cadences

* For a given piece (since this tool works for one composition at a time), you can check the **harmonic modules** (for example `7_Held, 6_-2, 8` for a cantizans-tenorizans pair) at any given point in your piece
    * **Import the piece** (just ONE at a time)
    * Run the **"missed cadence finder"** (Section D) above.  You must run this with just ONE piece!
* The default **modular ngram** is 3 events long.  Adjust **`n`** as needed for longer modules
* The default **interval type** is diatonic ('d').  Adjust **`kind`** as needed for chromatic ('c')

Use the results to report the new cadential voice functions and label, as explained above.

In [ ]:
piece_har = piece.getHarmonic(kind='d', compound=False)
ngrams = piece.getNgrams(df=piece_har, n=3, how='modules', exclude=['Rest'], offsets='last')
ngrams_filtered = ngrams.loc[result_offset_list,:].fillna("-")
ngrams_filtered

### C.5.  Inventory of Cadences Found:  One or Many Pieces at Once

* Here you can report an inventory of cadences by **type** and **tone** (and **evaded** status)


* To search multiple pieces at once (each returning its own dataframe of results):  enter the urls, separated by commas (each url within **single quotation marks**, and separated from the previous by a **comma** ).  Thus:

* **`corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0019.mei', 'https://crimproject.org/mei/CRIM_Model_0010.mei'])`**

In [ ]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0025.mei', 'https://crimproject.org/mei/CRIM_Model_0008.mei'])
list_of_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=False)
summary = combined_df.groupby(['CadType', 'Tone', 'Evaded']).size().reset_index(name='counts')
col_list = ['Composer', 'Title', 'Measure', 'Beat', 'CadType', 'Tone','Evaded','LeadingTones', 'Low','RelLow','RelTone','Progress','SinceLast','ToNext']
combined_df = combined_df[col_list]
combined_df

A simpler summary, without tone or evaded status:

In [ ]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0014_3.mei',
                             'https://crimproject.org/mei/CRIM_Model_0009.mei'])
list_of_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=False)
combined_df = pd.concat(list_of_dfs, ignore_index=True)
# Get the number of each type of cadence observed:
cadTypeCounts = combined_df['CadType'].value_counts()
print(cadTypeCounts)
# Get the number of cadences per Beat level:
cadBeatCounts = combined_df['Beat'].value_counts()
print(cadBeatCounts)


### Each Piece as A Single File


In [81]:
mass_list = ["CRIM_Mass_0001_1", "CRIM_Mass_0001_2", "CRIM_Mass_0001_3", "CRIM_Mass_0001_4",
 "CRIM_Mass_0001_5", "CRIM_Mass_0002_1", "CRIM_Mass_0002_2", "CRIM_Mass_0002_3", "CRIM_Mass_0002_4",
  "CRIM_Mass_0002_5", "CRIM_Mass_0003_1", "CRIM_Mass_0003_2", "CRIM_Mass_0003_3", "CRIM_Mass_0003_4",
   "CRIM_Mass_0003_5", "CRIM_Mass_0004_1", "CRIM_Mass_0004_2", "CRIM_Mass_0004_4",
    "CRIM_Mass_0004_5", "CRIM_Mass_0005_1", "CRIM_Mass_0005_2", "CRIM_Mass_0005_3", "CRIM_Mass_0005_4",
     "CRIM_Mass_0005_5", "CRIM_Mass_0006_1", "CRIM_Mass_0006_2", "CRIM_Mass_0006_3", "CRIM_Mass_0006_4", 
             "CRIM_Mass_0006_5", "CRIM_Mass_0007_1", "CRIM_Mass_0007_2", "CRIM_Mass_0007_3", 
             "CRIM_Mass_0007_4", "CRIM_Mass_0007_5", "CRIM_Mass_0008_1", "CRIM_Mass_0008_2", 
             "CRIM_Mass_0008_3", "CRIM_Mass_0008_4", "CRIM_Mass_0008_5", "CRIM_Mass_0009_1", 
             "CRIM_Mass_0009_2", "CRIM_Mass_0009_3", "CRIM_Mass_0009_4", "CRIM_Mass_0009_5", 
             "CRIM_Mass_0010_1", "CRIM_Mass_0010_2", "CRIM_Mass_0010_3",
     "CRIM_Mass_0010_4", "CRIM_Mass_0010_5", "CRIM_Mass_0011_1", "CRIM_Mass_0011_2", "CRIM_Mass_0011_3",
     "CRIM_Mass_0011_4", "CRIM_Mass_0011_5", "CRIM_Mass_0012_1", "CRIM_Mass_0012_2", "CRIM_Mass_0012_3",
      "CRIM_Mass_0012_4", "CRIM_Mass_0012_5", "CRIM_Mass_0013_1", "CRIM_Mass_0013_2", "CRIM_Mass_0013_3",
       "CRIM_Mass_0013_4", "CRIM_Mass_0013_5", "CRIM_Mass_0014_1", "CRIM_Mass_0014_2", "CRIM_Mass_0014_3",
        "CRIM_Mass_0014_4", "CRIM_Mass_0014_5", "CRIM_Mass_0015_1", "CRIM_Mass_0015_2", "CRIM_Mass_0015_3",
         "CRIM_Mass_0015_4", "CRIM_Mass_0015_5", "CRIM_Mass_0016_1", "CRIM_Mass_0016_2", "CRIM_Mass_0016_3", 
             "CRIM_Mass_0016_4", "CRIM_Mass_0016_5", "CRIM_Mass_0017_1", "CRIM_Mass_0017_2", "CRIM_Mass_0017_3", 
             "CRIM_Mass_0017_4", "CRIM_Mass_0017_5", "CRIM_Mass_0018_1", "CRIM_Mass_0018_2", "CRIM_Mass_0018_3", 
             "CRIM_Mass_0018_4", "CRIM_Mass_0018_5", "CRIM_Mass_0019_1", "CRIM_Mass_0019_2", "CRIM_Mass_0019_3", 
             "CRIM_Mass_0019_4", "CRIM_Mass_0019_5", "CRIM_Mass_0020_1", "CRIM_Mass_0020_2", "CRIM_Mass_0020_3", 
             "CRIM_Mass_0020_4", "CRIM_Mass_0020_5", "CRIM_Mass_0021_1", "CRIM_Mass_0021_2",
          "CRIM_Mass_0021_3", "CRIM_Mass_0021_4", "CRIM_Mass_0021_5", "CRIM_Mass_0022_2", "CRIM_Mass_0023_1", 
             "CRIM_Mass_0023_2", "CRIM_Mass_0023_3", "CRIM_Mass_0023_4", "CRIM_Mass_0023_5", "CRIM_Mass_0024_1", 
             "CRIM_Mass_0024_2", "CRIM_Mass_0024_3", "CRIM_Mass_0024_4", "CRIM_Mass_0024_5", "CRIM_Mass_0025_1", 
             "CRIM_Mass_0025_2", "CRIM_Mass_0025_3", "CRIM_Mass_0025_4", "CRIM_Mass_0025_5", "CRIM_Mass_0026_1", 
             "CRIM_Mass_0026_2", "CRIM_Mass_0026_3", "CRIM_Mass_0026_4", "CRIM_Mass_0026_5", "CRIM_Mass_0027_1", 
             "CRIM_Mass_0027_2", "CRIM_Mass_0027_3", "CRIM_Mass_0027_4", "CRIM_Mass_0027_5", "CRIM_Mass_0028_1", 
             "CRIM_Mass_0028_2", "CRIM_Mass_0028_3", "CRIM_Mass_0028_4", "CRIM_Mass_0028_5", "CRIM_Mass_0029_1", 
             "CRIM_Mass_0029_2", "CRIM_Mass_0029_3", "CRIM_Mass_0029_5", "CRIM_Mass_0030_1", 
             "CRIM_Mass_0030_2", "CRIM_Mass_0030_3", "CRIM_Mass_0030_4", "CRIM_Mass_0030_5", "CRIM_Mass_0031_1", "CRIM_Mass_0031_2", "CRIM_Mass_0031_3", "CRIM_Mass_0031_4", "CRIM_Mass_0031_5", "CRIM_Mass_0032_1", "CRIM_Mass_0032_2", "CRIM_Mass_0032_3", "CRIM_Mass_0032_4",
          "CRIM_Mass_0032_5", "CRIM_Mass_0033_1", "CRIM_Mass_0033_2", "CRIM_Mass_0033_3", "CRIM_Mass_0033_5", 
             "CRIM_Mass_0038_1", "CRIM_Mass_0038_2", "CRIM_Mass_0038_3", "CRIM_Mass_0038_4", "CRIM_Mass_0038_5", 
             "CRIM_Mass_0039_1", "CRIM_Mass_0039_2", "CRIM_Mass_0039_3", "CRIM_Mass_0039_4", "CRIM_Mass_0039_5", 
             "CRIM_Mass_0042_1", "CRIM_Mass_0042_2", "CRIM_Mass_0042_3", "CRIM_Mass_0042_4", "CRIM_Mass_0042_5", 
             "CRIM_Mass_0043_1", "CRIM_Mass_0043_2", "CRIM_Mass_0043_3", "CRIM_Mass_0043_4", "CRIM_Mass_0043_5", 
             "CRIM_Mass_0044_1", "CRIM_Mass_0044_2", "CRIM_Mass_0044_3", "CRIM_Mass_0044_4", "CRIM_Mass_0044_5", 
             "CRIM_Mass_0045_1", "CRIM_Mass_0045_2", "CRIM_Mass_0045_3", "CRIM_Mass_0045_4", "CRIM_Mass_0045_5", 
             "CRIM_Mass_0046_1", "CRIM_Mass_0046_2", "CRIM_Mass_0046_3", "CRIM_Mass_0046_4", "CRIM_Mass_0046_5", 
             "CRIM_Mass_0047_1", "CRIM_Mass_0047_2", "CRIM_Mass_0047_3", "CRIM_Mass_0047_4", "CRIM_Mass_0047_5", 
             "CRIM_Mass_0048_1", "CRIM_Mass_0048_2", "CRIM_Mass_0048_3", "CRIM_Mass_0048_4", "CRIM_Mass_0048_5", 
             "CRIM_Mass_0049_1",
          "CRIM_Mass_0049_3", "CRIM_Mass_0049_4"]

model_list = ["CRIM_Model_0001", "CRIM_Model_0002", "CRIM_Model_0008", "CRIM_Model_0009", "CRIM_Model_0010",
 "CRIM_Model_0011", "CRIM_Model_0012", "CRIM_Model_0013", "CRIM_Model_0014", "CRIM_Model_0015",
 "CRIM_Model_0016", "CRIM_Model_0017", "CRIM_Model_0019", "CRIM_Model_0020",
 "CRIM_Model_0021", "CRIM_Model_0023", "CRIM_Model_0025", "CRIM_Model_0026",
"CRIM_Model_0027", "CRIM_Model_0029", "CRIM_Model_0031", "CRIM_Model_0032",
"CRIM_Model_0033", "CRIM_Model_0036", "CRIM_Model_0037", "CRIM_Model_0038", "CRIM_Model_0039",
"CRIM_Model_0040", "CRIM_Model_0041", "CRIM_Model_0043", "CRIM_Model_0046",
"CRIM_Model_0047", "CRIM_Model_0048", "CRIM_Model_0049", "CRIM_Model_0050"]

In [82]:
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'
mei = '.mei'
for work in mass_list:
    file_url = git_prefix + work + mei 

    piece = importScore(file_url)
    output = piece.classifyCadences()

    output['Composer'] = piece.metadata["composer"]
    output['Title'] = piece.metadata["title"]
    output['Validation'] = ""
    output['Comments'] = ""
    col_list = ['Composer', 'Title', 'Measure', 'Beat', 'CadType', 'Tone','Evaded',
                'LeadingTones', 'Low','RelLow','RelTone',
                'Progress','SinceLast','ToNext', 'Validation', 'Comments']
    output = output[col_list]
    output.to_csv("saved_csv/" + work + "_Cadences_list" + ".csv")

Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.


### Entire Corpus as One List of Cadences

In [2]:
piece_list = ["CRIM_Mass_0001_1", "CRIM_Mass_0001_2", "CRIM_Mass_0001_3", "CRIM_Mass_0001_4", "CRIM_Mass_0001_5", "CRIM_Mass_0002_1", "CRIM_Mass_0002_2", "CRIM_Mass_0002_3", "CRIM_Mass_0002_4", "CRIM_Mass_0002_5", "CRIM_Mass_0003_1", "CRIM_Mass_0003_2", "CRIM_Mass_0003_3", "CRIM_Mass_0003_4", "CRIM_Mass_0003_5", "CRIM_Mass_0004_1", "CRIM_Mass_0004_2", "CRIM_Mass_0004_4", "CRIM_Mass_0004_5", "CRIM_Mass_0005_1", "CRIM_Mass_0005_2", "CRIM_Mass_0005_3", "CRIM_Mass_0005_4", "CRIM_Mass_0005_5", "CRIM_Mass_0006_1", "CRIM_Mass_0006_2", "CRIM_Mass_0006_3", "CRIM_Mass_0006_4", "CRIM_Mass_0006_5", "CRIM_Mass_0007_1", "CRIM_Mass_0007_2", "CRIM_Mass_0007_3", "CRIM_Mass_0007_4", "CRIM_Mass_0008_1", "CRIM_Mass_0008_2", "CRIM_Mass_0008_3", "CRIM_Mass_0008_4", "CRIM_Mass_0008_5", "CRIM_Mass_0009_1", "CRIM_Mass_0009_2", "CRIM_Mass_0009_3", "CRIM_Mass_0009_4", "CRIM_Mass_0009_5", "CRIM_Mass_0010_1", "CRIM_Mass_0010_2", "CRIM_Mass_0010_3", "CRIM_Mass_0010_4", "CRIM_Mass_0010_5", "CRIM_Mass_0011_1", "CRIM_Mass_0011_2", "CRIM_Mass_0011_3", "CRIM_Mass_0011_4", "CRIM_Mass_0011_5", "CRIM_Mass_0012_1", "CRIM_Mass_0012_2", "CRIM_Mass_0012_3", "CRIM_Mass_0012_4", "CRIM_Mass_0012_5", "CRIM_Mass_0013_1", "CRIM_Mass_0013_2", "CRIM_Mass_0013_3", "CRIM_Mass_0013_4", "CRIM_Mass_0013_5", "CRIM_Mass_0014_1", "CRIM_Mass_0014_2", "CRIM_Mass_0014_3", "CRIM_Mass_0014_4", "CRIM_Mass_0014_5", "CRIM_Mass_0015_1", "CRIM_Mass_0015_2", "CRIM_Mass_0015_3", "CRIM_Mass_0015_4", "CRIM_Mass_0015_5", "CRIM_Mass_0016_1", "CRIM_Mass_0016_2", "CRIM_Mass_0016_3", "CRIM_Mass_0016_4", "CRIM_Mass_0016_5", "CRIM_Mass_0017_1", "CRIM_Mass_0017_2", "CRIM_Mass_0017_3", "CRIM_Mass_0017_4", "CRIM_Mass_0017_5", "CRIM_Mass_0018_1", "CRIM_Mass_0018_2", "CRIM_Mass_0018_3", "CRIM_Mass_0018_4", "CRIM_Mass_0018_5", "CRIM_Mass_0019_1", "CRIM_Mass_0019_2", "CRIM_Mass_0019_3", "CRIM_Mass_0019_4", "CRIM_Mass_0019_5", "CRIM_Mass_0020_1", "CRIM_Mass_0020_2", "CRIM_Mass_0020_3", "CRIM_Mass_0020_4", "CRIM_Mass_0020_5", "CRIM_Mass_0021_1", "CRIM_Mass_0021_2", "CRIM_Mass_0021_3", "CRIM_Mass_0021_4", "CRIM_Mass_0021_5", "CRIM_Mass_0022_2", "CRIM_Mass_0023_1", "CRIM_Mass_0023_2", "CRIM_Mass_0023_3", "CRIM_Mass_0023_4", "CRIM_Mass_0023_5", "CRIM_Mass_0024_1", "CRIM_Mass_0024_2", "CRIM_Mass_0024_3", "CRIM_Mass_0024_4", "CRIM_Mass_0024_5", "CRIM_Mass_0025_1", "CRIM_Mass_0025_2", "CRIM_Mass_0025_3", "CRIM_Mass_0025_4", "CRIM_Mass_0025_5", "CRIM_Mass_0026_1", "CRIM_Mass_0026_2", "CRIM_Mass_0026_3", "CRIM_Mass_0026_4", "CRIM_Mass_0026_5", "CRIM_Mass_0027_1", "CRIM_Mass_0027_2", "CRIM_Mass_0027_3", "CRIM_Mass_0027_4", "CRIM_Mass_0027_5", "CRIM_Mass_0028_1", "CRIM_Mass_0028_2", "CRIM_Mass_0028_3", "CRIM_Mass_0028_4", "CRIM_Mass_0028_5", "CRIM_Mass_0029_1", "CRIM_Mass_0029_2", "CRIM_Mass_0029_3", "CRIM_Mass_0029_5", "CRIM_Mass_0030_1", "CRIM_Mass_0030_2", "CRIM_Mass_0030_3", "CRIM_Mass_0030_4", "CRIM_Mass_0030_5", "CRIM_Mass_0031_1", "CRIM_Mass_0031_2", "CRIM_Mass_0031_3", "CRIM_Mass_0031_4", "CRIM_Mass_0031_5", "CRIM_Mass_0032_1", "CRIM_Mass_0032_2", "CRIM_Mass_0032_3", "CRIM_Mass_0032_4", "CRIM_Mass_0032_5", "CRIM_Mass_0033_1", "CRIM_Mass_0033_2", "CRIM_Mass_0033_3", "CRIM_Mass_0033_5", "CRIM_Mass_0036_1", "CRIM_Mass_0036_2", "CRIM_Mass_0036_3", "CRIM_Mass_0036_4", "CRIM_Mass_0036_5", "CRIM_Mass_0038_1", "CRIM_Mass_0038_2", "CRIM_Mass_0038_3", "CRIM_Mass_0038_4", "CRIM_Mass_0038_5", "CRIM_Mass_0039_1", "CRIM_Mass_0039_2", "CRIM_Mass_0039_3", "CRIM_Mass_0039_4", "CRIM_Mass_0039_5", "CRIM_Mass_0043_1", "CRIM_Mass_0043_2", "CRIM_Mass_0043_3", "CRIM_Mass_0043_4", "CRIM_Mass_0043_5", "CRIM_Mass_0044_1", "CRIM_Mass_0044_2", "CRIM_Mass_0044_3", "CRIM_Mass_0044_4", "CRIM_Mass_0044_5", "CRIM_Mass_0045_1", "CRIM_Mass_0045_2", "CRIM_Mass_0045_3", "CRIM_Mass_0045_4", "CRIM_Mass_0045_5", "CRIM_Mass_0046_1", "CRIM_Mass_0046_2", "CRIM_Mass_0046_3", "CRIM_Mass_0046_4", "CRIM_Mass_0046_5", "CRIM_Mass_0047_1", "CRIM_Mass_0047_2", "CRIM_Mass_0047_3", "CRIM_Mass_0047_4", "CRIM_Mass_0047_5", "CRIM_Mass_0048_1", "CRIM_Mass_0048_2", "CRIM_Mass_0048_3", "CRIM_Mass_0048_4", "CRIM_Mass_0048_5", "CRIM_Mass_0049_1", "CRIM_Mass_0049_3", "CRIM_Mass_0049_4", "CRIM_Model_0001", "CRIM_Model_0002", "CRIM_Model_0008", "CRIM_Model_0009", "CRIM_Model_0010", "CRIM_Model_0011", "CRIM_Model_0012", "CRIM_Model_0013", "CRIM_Model_0014", "CRIM_Model_0015", "CRIM_Model_0016", "CRIM_Model_0017", "CRIM_Model_0019", "CRIM_Model_0020", "CRIM_Model_0021", "CRIM_Model_0023", "CRIM_Model_0025", "CRIM_Model_0026", "CRIM_Model_0027", "CRIM_Model_0029", "CRIM_Model_0031", "CRIM_Model_0032", "CRIM_Model_0033", "CRIM_Model_0036", "CRIM_Model_0037", "CRIM_Model_0038", "CRIM_Model_0039", "CRIM_Model_0040", "CRIM_Model_0041", "CRIM_Model_0043", "CRIM_Model_0045", "CRIM_Model_0046", "CRIM_Model_0047", "CRIM_Model_0048", "CRIM_Model_0049", "CRIM_Model_0050"]

In [3]:
big_list = []
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'
mei = '.mei'
for piece in piece_list:
    url = git_prefix + piece + mei
    big_list.append(url)

In [4]:
corpus = CorpusBase(big_list)
list_of_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=True)
# Get the number of each type of cadence observed:
cadTypeCounts = combined_df['CadType'].value_counts()
# print(cadTypeCounts)
# Get the number of cadences per Beat level:
cadBeatCounts = combined_df['Beat'].value_counts()
# print(cadBeatCounts)

Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_1.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_2.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_3.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_4.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_5.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0002_1.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0002_2.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/

mei.base: WARNING: Importing <slur> without @startid and @endid is not yet supported.


Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0005_3.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0005_4.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0005_5.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0006_1.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0006_2.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0006_3.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0006_4.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/

Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_5.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0016_1.mei


mei.base: WARNING: Importing <tie> without @startid and @endid is not yet supported.


...

This method has been deprecated. Please use .cvfs() for cadential
        voice function analysis or .cadences() for cadential analysis. In both cases,
        you no longer need the `return_type` parameter and should not pass it.
        The `keep_keys` parameter works the same in those two functions.
This method has been deprecated. Please use .cvfs() for cadential
        voice function analysis or .cadences() for cadential analysis. In both cases,
        you no longer need the `return_type` parameter and should not pass it.
        The `keep_keys` parameter works the same in those two functions.
This method has been deprecated. Please use .cvfs() for cadential
        voice function analysis or .cadences() for cadential analysis. In both cases,
        you no longer need the `return_type` parameter and should not pass it.
        The `keep_keys` parameter works the same in those two functions.
This method has been deprecated. Please use .cvfs() for cadential
        voice function

This method has been deprecated. Please use .cvfs() for cadential
        voice function analysis or .cadences() for cadential analysis. In both cases,
        you no longer need the `return_type` parameter and should not pass it.
        The `keep_keys` parameter works the same in those two functions.
This method has been deprecated. Please use .cvfs() for cadential
        voice function analysis or .cadences() for cadential analysis. In both cases,
        you no longer need the `return_type` parameter and should not pass it.
        The `keep_keys` parameter works the same in those two functions.
This method has been deprecated. Please use .cvfs() for cadential
        voice function analysis or .cadences() for cadential analysis. In both cases,
        you no longer need the `return_type` parameter and should not pass it.
        The `keep_keys` parameter works the same in those two functions.
This method has been deprecated. Please use .cvfs() for cadential
        voice function

This method has been deprecated. Please use .cvfs() for cadential
        voice function analysis or .cadences() for cadential analysis. In both cases,
        you no longer need the `return_type` parameter and should not pass it.
        The `keep_keys` parameter works the same in those two functions.
This method has been deprecated. Please use .cvfs() for cadential
        voice function analysis or .cadences() for cadential analysis. In both cases,
        you no longer need the `return_type` parameter and should not pass it.
        The `keep_keys` parameter works the same in those two functions.
This method has been deprecated. Please use .cvfs() for cadential
        voice function analysis or .cadences() for cadential analysis. In both cases,
        you no longer need the `return_type` parameter and should not pass it.
        The `keep_keys` parameter works the same in those two functions.
This method has been deprecated. Please use .cvfs() for cadential
        voice function

This method has been deprecated. Please use .cvfs() for cadential
        voice function analysis or .cadences() for cadential analysis. In both cases,
        you no longer need the `return_type` parameter and should not pass it.
        The `keep_keys` parameter works the same in those two functions.
This method has been deprecated. Please use .cvfs() for cadential
        voice function analysis or .cadences() for cadential analysis. In both cases,
        you no longer need the `return_type` parameter and should not pass it.
        The `keep_keys` parameter works the same in those two functions.
This method has been deprecated. Please use .cvfs() for cadential
        voice function analysis or .cadences() for cadential analysis. In both cases,
        you no longer need the `return_type` parameter and should not pass it.
        The `keep_keys` parameter works the same in those two functions.
This method has been deprecated. Please use .cvfs() for cadential
        voice function

Clausula Vera          2662
Authentic              1831
Phrygian                437
Altizans Only            74
Plagal                   32
Phrygian Altizans         4
Double Leading Tone       2
Name: CadType, dtype: int64
1.000000    3467
3.000000    1724
2.000000     246
4.000000     166
1.500000      41
2.500000      39
4.500000      38
3.500000      29
1.250000       4
2.750000       4
5.000000       3
1.333333       2
1.750000       1
Name: Beat, dtype: int64


In [6]:
cadTypeCounts


Clausula Vera          2662
Authentic              1831
Phrygian                437
Altizans Only            74
Plagal                   32
Phrygian Altizans         4
Double Leading Tone       2
Name: CadType, dtype: int64

In [5]:
combined_df.to_csv("saved_csv/all_cads.csv")

In [57]:
mass_sets = [('CRIM_Mass_0015',
('https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0017.mei',
 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_1.mei',
 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_2.mei',
 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_3.mei',
 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_4.mei',
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_5.mei'))]

# git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# list(mass[1])

In [58]:
cad_combined = pd.DataFrame()

col_list = ['Composer', 'Title', 'Measure', 'Beat', 'CadType', 'Tone','Evaded','LeadingTones', 'Low','RelLow','RelTone','Progress','SinceLast','ToNext']
for mass in mass_sets:
    csv_name = 'saved_csv/' + mass_sets[0][0] + '_' + 'Cadences' + '.csv'
    url_list = list(mass[1])
    corpus = CorpusBase(url_list)
    list_of_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=True)
    combined_df = pd.concat(list_of_dfs, ignore_index=True)
    combined_df = combined_df[col_list]
    combined_df["Validation"] = ''
    combined_df["Comments"] = ''
    combined_df.to_csv(csv_name)

Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0017.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_1.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_2.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_3.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_4.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_5.mei


In [59]:
combined_df

,Composer,Title,Measure,Beat,CadType,Tone,Evaded,LeadingTones,Low,RelLow,RelTone,Progress,SinceLast,ToNext,Validation,Comments
0,Josquin Des Prés,Benedicta es,13,3.0,Clausula Vera,G,False,1.0,G2,P1,P1,0.066845,100.0,12.0,,
1,Josquin Des Prés,Benedicta es,15,1.0,Clausula Vera,D,False,0.0,D3,P5,-P4,0.074866,12.0,8.0,,
2,Josquin Des Prés,Benedicta es,16,1.0,Clausula Vera,G,False,1.0,E3,M6,P1,0.080214,8.0,12.0,,
3,Josquin Des Prés,Benedicta es,17,3.0,Clausula Vera,G,False,0.0,G2,P1,P1,0.088235,12.0,12.0,,
4,Josquin Des Prés,Benedicta es,19,1.0,Clausula Vera,D,False,1.0,D3,P5,-P4,0.096257,12.0,8.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Giovanni Pierluigi da Palestrina,Missa Benedicta es: Agnus Dei,72,1.0,Authentic,A,True,0.0,C3,P4,M2,0.647541,72.0,52.0,,
213,Giovanni Pierluigi da Palestrina,Missa Benedicta es: Agnus Dei,77,1.0,Authentic,G,False,0.0,G2,P1,P1,0.700820,52.0,28.0,,
214,Giovanni Pierluigi da Palestrina,Missa Benedicta es: Agnus Dei,79,1.0,Authentic,NaN,True,-1.0,G3,P8,NaN,0.729508,28.0,32.0,,
215,Giovanni Pierluigi da Palestrina,Missa Benedicta es: Agnus Dei,82,1.0,NaN,D,NaN,NaN,D3,P5,-P4,0.762295,32.0,160.0,,


In [29]:
cadences

,Measure,Beat,CadType,Tone,Evaded,LeadingTones,Low,RelLow,RelTone,Progress,SinceLast,ToNext
24.0,4,1.0,Authentic,G,False,1,G3,P8,P1,0.050000,24.0,16.0
40.0,6,1.0,Authentic,G,False,1,G2,P1,P1,0.083333,16.0,80.0
120.0,16,1.0,Authentic,D,False,1,D3,P5,-P4,0.250000,80.0,36.0
156.0,20,3.0,Authentic,B-,False,1,B-2,m3,m3,0.325000,36.0,52.0
208.0,27,1.0,Authentic,G,False,1,G2,P1,P1,0.433333,52.0,51.0
259.0,33,2.5,Authentic,G,False,1,G2,P1,P1,0.539583,51.0,125.0
384.0,49,1.0,Clausula Vera,D,False,1,D3,P5,-P4,0.800000,125.0,36.0
420.0,53,3.0,Authentic,D,False,1,D3,P5,-P4,0.875000,36.0,28.0
448.0,57,1.0,Authentic,G,False,1,G3,P8,P1,0.933333,28.0,16.0
464.0,59,1.0,Authentic,G,False,1,G2,P1,P1,0.966667,16.0,16.0
